<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_LLM_Math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
OPENAI_API_KEY = 'your OpenAI API key here'

In [ ]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
# llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-3.5-turbo")

In [ ]:
from langchain.tools import BaseTool

class EvaluateMathExpression(BaseTool):
    name = "Math Evaluation"
    description = 'use this tool to evaluate a math expression.'

    def _run(self, expr: str):
        return eval(expr)
    
    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet")

tools = [EvaluateMathExpression()]

In [ ]:
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
    )
)

In [ ]:
for message in agent.agent.llm_chain.prompt.messages:
  print(message)

In [ ]:
agent(f"What is 2 * 2 * 0.13 - 1.001?")

In [ ]:
2 * 2 * 0.13 - 1.001

In [ ]:
from langchain.agents.conversational_chat.prompt import (PREFIX)
system_message = PREFIX + "\n" + '''
Unfortunately, Assistant is terrible at maths. Assistant should always refers to available tools and never try to answer math questions by itself
'''

In [ ]:
new_prompt = agent.agent.create_prompt(
    system_message=system_message,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [ ]:
agent(f"What is 2 * 2 * 0.13 - 1.001?")

In [ ]:
agent.memory.buffer